In [2]:
!nvidia-smi

Tue Apr 20 12:54:11 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 450.36.06    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro P5000        On   | 00000000:00:05.0 Off |                  Off |
| 26%   27C    P8     6W / 180W |      1MiB / 16278MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import pandas as pd
import torch
from torch import Tensor
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.init import xavier_uniform_
from transformers import BertTokenizer, BertForSequenceClassification, GPT2ForSequenceClassification, GPT2Tokenizer
import torch.optim as optim
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from torchtext.legacy.data import BucketIterator, TabularDataset, Iterator, Field, LabelField

root=''

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# device = torch.device('cpu')
print('Using PyTorch version:', torch.__version__, ' Device:', device)

df_train = pd.read_csv(root + 'train2.csv')
df_train.head()

Using PyTorch version: 1.8.1+cu102  Device: cuda


,text,label
0,Буш: Американскую экономику спасут этические с...,0
1,"Как правильно мерить член, чтобы получилось 20...",1
2,По английски мальчик наоборот будет йоб.,1
3,Употребление спортсменами допинга сразу повлеч...,0
4,В Петербурге поймали дезертира,0


In [3]:
# tokenizer = BertTokenizer.from_pretrained(bert_root+'rubert-base')
tokenizer = GPT2Tokenizer.from_pretrained('sberbank-ai/rugpt3medium_based_on_gpt2')
tokenizer.add_special_tokens({'pad_token': '<pad>'})


MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

text_field = Field(use_vocab=False, tokenize=tokenizer.encode, lower=False,
                   include_lengths=False, pad_token=PAD_INDEX, unk_token=UNK_INDEX,  fix_length=MAX_SEQ_LEN)
label_field = Field(sequential=False, use_vocab=False, batch_first=True, dtype=torch.float)

fields = [('text', text_field), ('label', label_field)]

# TabularDataset

train, valid, test = TabularDataset.splits(path='.', train=root+'train2.csv', validation=root+'val2.csv',
                                           test=root+'test2.csv', format='CSV', fields=fields, skip_header=True)

# Iterators

train_iter = BucketIterator(train, batch_size=16, sort_key=lambda x: len(x.text),
                            device=device, train=True, sort=True, sort_within_batch=True)
valid_iter = BucketIterator(valid, batch_size=16, sort_key=lambda x: len(x.text),
                            device=device, train=True, sort=True, sort_within_batch=True)
test_iter = Iterator(test, batch_size=1, device=device, train=False, shuffle=False, sort=False)

In [5]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# print(f'The model has {count_parameters(model):,} trainable parameters')

## 4. Обучение модели.

Обученную модель примените для получения предсказаний на Kaggle. Баллы за задания из ноутбука (максимум 5) ставятся в случае, если модель корректно отработала, и вы смогли успешно отправить посылку на Kaggle.

In [6]:
class BERT(nn.Module):

    def __init__(self):
        super(BERT, self).__init__()

        options_name = "rubert-base-cased"
        self.encoder = BertForSequenceClassification.from_pretrained('DeepPavlov/rubert-base-cased', num_labels=2)

    def forward(self, text, label):
        loss, text_fea = self.encoder(text, labels=label)[:2]

        return loss, text_fea

class GPT(nn.Module):

    def __init__(self):
        super(GPT, self).__init__()

        options_name = "sberbank-ai/rugpt3medium_based_on_gpt2"
        self.encoder = GPT2ForSequenceClassification.from_pretrained(options_name, num_labels=2)
        self.encoder.resize_token_embeddings(len(tokenizer))
        self.encoder.config.pad_token_id = PAD_INDEX

    def forward(self, text, label):
        loss, text_fea = self.encoder(text, labels=label)[:2]

        return loss, text_fea

In [4]:
# Save and Load Functions

def save_checkpoint(save_path, model, valid_loss):

    if save_path == None:
        return
    
    state_dict = {'model_state_dict': model.state_dict(),
                  'valid_loss': valid_loss}
    
    torch.save(state_dict, save_path)
    print(f'Model saved to ==> {save_path}')

def load_checkpoint(load_path, model):
    
    if load_path==None:
        return
    
    state_dict = torch.load(load_path, map_location=device)
    print(f'Model loaded from <== {load_path}')
    
    model.load_state_dict(state_dict['model_state_dict'])
    return state_dict['valid_loss']


def save_metrics(save_path, train_loss_list, valid_loss_list, global_steps_list):

    if save_path == None:
        return
    
    state_dict = {'train_loss_list': train_loss_list,
                  'valid_loss_list': valid_loss_list,
                  'global_steps_list': global_steps_list}
    
    torch.save(state_dict, save_path)
    print(f'Model saved to ==> {save_path}')


def load_metrics(load_path):

    if load_path==None:
        return
    
    state_dict = torch.load(load_path, map_location=device)
    print(f'Model loaded from <== {load_path}')
    
    return state_dict['train_loss_list'], state_dict['valid_loss_list'], state_dict['global_steps_list']

In [31]:
import neptune
neptune.init(project_qualified_name='rototo/jokes', api_token='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJhOTcwMzgwYi05NjE5LTQ1OTQtYWQxZC1lZTRiMDk5YzgyOGQifQ==')

Project(rototo/jokes)

In [32]:
neptune.create_experiment('bert-vs-gen-large classification')

https://app.neptune.ai/rototo/jokes/e/JOK-4


Experiment(JOK-4)

In [23]:
from tqdm import trange, tqdm
def train(model,
          optimizer,
          criterion = nn.BCELoss(),
          train_loader = train_iter,
          valid_loader = valid_iter,
          num_epochs = 5,
          eval_every = len(train_iter) // 2,
          file_path = None,
          best_valid_loss = float("Inf")):
    
    # initialize running values
    running_loss = 0.0
    valid_running_loss = 0.0
    global_step = 0
    train_loss_list = []
    valid_loss_list = []
    global_steps_list = []

    # training loop
    model.train()
    for epoch in trange(num_epochs):
        for batch in tqdm(train_loader, position=0, leave=True):
            labels = batch.label.type(torch.LongTensor)           
            labels = labels.to(device)
            text = batch.text.type(torch.LongTensor)  
            text = text.to(device)
            output = model(text.transpose(0, 1), labels)
            loss, _ = output

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # update running values
            running_loss += loss.item()
            global_step += 1
            neptune.log_metric('Train loss', running_loss / global_step)

            # evaluation step
            if global_step % eval_every == 0:
                y_pred = []
                y_true = []
                model.eval()
                with torch.no_grad():                    

                    # validation loop
                    for batch in valid_loader:
                        labels = batch.label.type(torch.LongTensor)           
                        labels = labels.to(device)
                        text = batch.text.type(torch.LongTensor)  
                        text = text.to(device)
                        output = model(text.transpose(0, 1), labels)
                        loss, output = output
                        y_pred.extend(torch.argmax(output, 1).tolist())
                        y_true.extend(labels.tolist())
                        
                        valid_running_loss += loss.item()

                # evaluation
                average_train_loss = running_loss / eval_every
                average_valid_loss = valid_running_loss / len(valid_loader)
                train_loss_list.append(average_train_loss)
                valid_loss_list.append(average_valid_loss)
                global_steps_list.append(global_step)
                val_acc = accuracy_score(y_true, y_pred)

                # resetting running values
                running_loss = 0.0                
                valid_running_loss = 0.0
                model.train()

                # print progress
                print('Epoch [{}/{}], Step [{}/{}], Train Loss: {:.4f}, Valid Loss: {:.4f}, Valid ACC: {:.4f}'
                      .format(epoch+1, num_epochs, global_step, num_epochs*len(train_loader),
                              average_train_loss, average_valid_loss, val_acc))
                
                # checkpoint
                if best_valid_loss > average_valid_loss:
                    best_valid_loss = average_valid_loss
                    save_checkpoint(file_path + '/' + 'model.pt', model, best_valid_loss)
                    save_metrics(file_path + '/' + 'metrics.pt', train_loss_list, valid_loss_list, global_steps_list)
    
    save_metrics(file_path + '/' + 'metrics.pt', train_loss_list, valid_loss_list, global_steps_list)
    print('Finished Training!')

# model = BERT().to(device)
model2 = GPT().to(device)

optimizer = optim.Adam(model.parameters(), lr=2e-5)
# optimizer2 = optim.Adam(model2.parameters(), lr=2e-5)

In [11]:
# train(model=model, optimizer=optimizer, file_path=root+'saves')

In [12]:
train(model=model2, optimizer=optimizer2, file_path=root+'classification')

 29%|██▊       | 2934/10255 [47:55<2:00:05,  1.02it/s]Unexpected error in ping thread.
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py", line 706, in urlopen
    chunked=chunked,
  File "/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py", line 445, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py", line 440, in _make_request
    httplib_response = conn.getresponse()
  File "/usr/lib/python3.6/http/client.py", line 1373, in getresponse
    response.begin()
  File "/usr/lib/python3.6/http/client.py", line 311, in begin
    version, status, reason = self._read_status()
  File "/usr/lib/python3.6/http/client.py", line 272, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "/usr/lib/python3.6/socket.py", line 586, in readinto
    return self._sock.recv_into(b)
  File "/usr/li

Epoch [1/5], Step [5127/51275], Train Loss: 0.0074, Valid Loss: 0.0072, Valid ACC: 0.9982


 50%|████▉     | 5127/10255 [1:36:33<325:42:17, 228.65s/it]

Model saved to ==> classification/model.pt
Model saved to ==> classification/metrics.pt


100%|█████████▉| 10254/10255 [3:13:25<03:48, 228.26s/it]   

Epoch [1/5], Step [10254/51275], Train Loss: 0.0031, Valid Loss: 0.0741, Valid ACC: 0.9744


 20%|██        | 1/5 [3:13:36<12:54:27, 11616.84s/it]


KeyboardInterrupt: 

In [9]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

def evaluate(model, loader):
    y_pred = []
    y_true = []

    model.eval()
    with torch.no_grad():
        for batch in loader:
                labels = batch.label.type(torch.LongTensor)           
                labels = labels.to(device)
                text = batch.text.type(torch.LongTensor)  
                text = text.to(device)
                output = model(text.transpose(0, 1), labels)

                _, output = output
                y_pred.extend(torch.argmax(output, 1).tolist())
                y_true.extend(labels.tolist())

    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    pr, rec = precision_score(y_true, y_pred), recall_score(y_true, y_pred)

#     neptune.log_metric('accuracy', accuracy)
#     neptune.log_metric('f1-score', f1)
#     neptune.log_metric('precision', pr)
#     neptune.log_metric('recall', rec)
            
    
    print('Classification Report:')
    # wandb.log({'classification_report': classification_report(y_true, y_pred, digits=4)})
    print(classification_report(y_true, y_pred, digits=4))
    print('****\n Acc:', accuracy_score(y_true, y_pred))
    return y_true, y_pred

In [8]:
best_model = GPT().to(device)

load_checkpoint(root+'classification' + '/model.pt', best_model)

# evaluate(best_model, valid_iter)

Some weights of the model checkpoint at sberbank-ai/rugpt3medium_based_on_gpt2 were not used when initializing GPT2ForSequenceClassification: ['lm_head.weight']
- This IS expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at sberbank-ai/rugpt3medium_based_on_gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded from <== classification/model.pt


0.007222916210688032

In [11]:
y_true_test, y_pred_test = evaluate(best_model, test_iter)

Classification Report:
              precision    recall  f1-score   support

           0     0.9977    0.9985    0.9981     30221
           1     0.9978    0.9967    0.9973     21053

    accuracy                         0.9978     51274
   macro avg     0.9978    0.9976    0.9977     51274
weighted avg     0.9978    0.9978    0.9978     51274

****
 Acc: 0.9977571478722159


In [19]:
j = 0
wrong = []
for i, d in enumerate(test_iter):
    if y_true_test[i] != y_pred_test[i]:
        print(f'****\nTrue label: {int(d.label.item())} \n Шутка {i}: ', tokenizer.decode(d.text[:,0]).replace('<pad>',''))
        j += 1
    if j == 5:
        break

****
True label: 0 
 Шутка 48:  Муктада аль-Садр: Победа или смерть!
****
True label: 0 
 Шутка 102:  Новым генпрокурором станет нынешний руководитель аппарата правительства?
****
True label: 1 
 Шутка 954:  Трое неизвестных отобрали у прохожего паспорт и порвали. Неизвестных стало четверо
****
True label: 1 
 Шутка 1139:  Золушка: - Туфелька мне подошла, когда свадьба? Принц: - Это был полуфинал. Сейчас будем мерить бюстгальтер...
****
True label: 1 
 Шутка 1701:  Не будите во мне зверя! Ему ко второй паре.


## vs Generated

In [8]:
tokenizer = GPT2Tokenizer.from_pretrained('sberbank-ai/rugpt3medium_based_on_gpt2')
tokenizer.add_special_tokens({'pad_token': '<pad>'})
PAD_INDEX = tokenizer.pad_token_id
model = GPT().to(device)

Some weights of the model checkpoint at sberbank-ai/rugpt3medium_based_on_gpt2 were not used when initializing GPT2ForSequenceClassification: ['lm_head.weight']
- This IS expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at sberbank-ai/rugpt3medium_based_on_gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
load_checkpoint(root+'classification' + '/model.pt', model)

Model loaded from <== classification/model.pt


0.007222916210688032

In [10]:
MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)
root=''
text_field = Field(use_vocab=False, tokenize=tokenizer.encode, lower=False,
                   include_lengths=False, pad_token=PAD_INDEX, unk_token=UNK_INDEX,  fix_length=MAX_SEQ_LEN)
label_field = Field(sequential=False, use_vocab=False, batch_first=True, dtype=torch.float)

fields = [('text', text_field), ('label', label_field)]

# TabularDataset

train, valid, test = TabularDataset.splits(path='.', train=root+'train2.csv', validation=root+'val2.csv',
                                           test=root+'jokes_generated.csv', format='CSV', fields=fields, skip_header=True)

test_iter = Iterator(test, batch_size=1, device=device, train=False, shuffle=False, sort=False)

In [22]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

def evaluate2(model, loader):
    y_pred = []
    y_true = []

    model.eval()
    with torch.no_grad():
        for batch in loader:
                labels = batch.label.type(torch.LongTensor)           
                labels = labels.to(device)
                text = batch.text.type(torch.LongTensor)  
                text = text.to(device)
                output = model(text.transpose(0, 1), labels)

                _, output = output
                y_pred.extend(torch.argmax(output, 1).tolist())
                y_true.extend(labels.tolist())
            
    
    print('Classification Report:')
    # wandb.log({'classification_report': classification_report(y_true, y_pred, digits=4)})
    print(classification_report(y_true, y_pred, digits=4))
    print('****\n Acc:', accuracy_score(y_true, y_pred))

evaluate2(model, test_iter)

### Trained on not-Gen vs Gen SVM

In [4]:
import nltk
nltk.download("stopwords")
#--------#

from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation

#Create lemmatizer and stopwords list
mystem = Mystem() 
russian_stopwords = stopwords.words("russian")

#Preprocess function
def preprocess_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]
    
    text = " ".join(tokens)
    
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


In [16]:
train_gen_jokes = pd.read_csv('train_gen_jokes.csv')
test_gen_jokes = pd.read_csv('test_gen_jokes.csv')

In [17]:
def prepare_text(text):
    text = text.replace('\n', '')
    return preprocess_text(text)

In [18]:
train_gen_jokes['text'] = train_gen_jokes['text'].apply(lambda x: prepare_text(x))

In [20]:
test_gen_jokes['text'] = test_gen_jokes['text'].apply(lambda x: prepare_text(x))

In [21]:
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

svc_pipe = Pipeline([('cv', CountVectorizer()), ('svc', SVC())])
svc_pipe.fit(train_gen_jokes['text'], train_gen_jokes['label'])

Pipeline(steps=[('cv', CountVectorizer()), ('svc', SVC())])

In [22]:
y_pred = svc_pipe.predict(test_gen_jokes['text'])
print(accuracy_score(test_gen_jokes['label'], y_pred))

0.7187981510015409


In [ ]:
from wordcloud import WordCloud

def draw_wordcloud(texts, max_words=1000, width=1000, height=500):
    wordcloud = WordCloud(background_color='white', max_words=max_words,
                          width=width, height=height)
    
    joint_texts = ' '.join(list(texts))
    wordcloud.generate(joint_texts)
    return wordcloud.to_image()

### Gen vs not-Gen BERT

In [21]:
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
# tokenizer = GPT2Tokenizer.from_pretrained('sberbank-ai/rugpt3medium_based_on_gpt2')
# tokenizer.add_special_tokens({'pad_token': '<pad>'})


MAX_SEQ_LEN = 64
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

text_field = Field(use_vocab=False, tokenize=tokenizer.encode, lower=False,
                   include_lengths=False, pad_token=PAD_INDEX, unk_token=UNK_INDEX,  fix_length=MAX_SEQ_LEN)
label_field = Field(sequential=False, use_vocab=False, batch_first=True, dtype=torch.float)

fields = [('text', text_field), ('label', label_field)]

# TabularDataset

train, valid, test = TabularDataset.splits(path='.', train=root+'train_gen_jokes.csv', validation=root+'val_gen_jokes.csv',
                                           test=root+'test_gen_jokes.csv', format='CSV', fields=fields, skip_header=True)

# Iterators

train_iter = BucketIterator(train, batch_size=16, sort_key=lambda x: len(x.text),
                            device=device, train=True, sort=True, sort_within_batch=True)
valid_iter = BucketIterator(valid, batch_size=16, sort_key=lambda x: len(x.text),
                            device=device, train=True, sort=True, sort_within_batch=True)
test_iter = Iterator(test, batch_size=1, device=device, train=False, shuffle=False, sort=False)

In [37]:
bert = BERT().to(device)
optimizer = optim.Adam(bert.parameters(), lr=2e-5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [38]:
train(model=bert, optimizer=optimizer, file_path='classification_gen', num_epochs=10)

 50%|████▉     | 323/649 [01:02<01:02,  5.18it/s]

Epoch [1/10], Step [324/6490], Train Loss: 0.5313, Valid Loss: 0.4620, Valid ACC: 0.7820


 50%|█████     | 325/649 [01:18<19:22,  3.59s/it]

Model saved to ==> classification_gen/model.pt
Model saved to ==> classification_gen/metrics.pt


  0%|          | 0/649 [00:00<?, ?it/s].43s/it]

...

Epoch [10/10], Step [6156/6490], Train Loss: 0.0214, Valid Loss: 0.5852, Valid ACC: 0.8544


 99%|█████████▊| 640/649 [02:33<00:29,  3.28s/it]

Epoch [10/10], Step [6480/6490], Train Loss: 0.0060, Valid Loss: 0.6479, Valid ACC: 0.8844


100%|██████████| 10/10 [25:57<00:00, 155.75s/it]

Model saved to ==> classification_gen/metrics.pt
Finished Training!


In [39]:
evaluate(bert, test_iter)

Classification Report:
              precision    recall  f1-score   support

           0     0.8464    0.9422    0.8917      1333
           1     0.9308    0.8195    0.8716      1263

    accuracy                         0.8825      2596
   macro avg     0.8886    0.8809    0.8817      2596
weighted avg     0.8874    0.8825    0.8819      2596

****
 Acc: 0.8825115562403698
